<a href="https://colab.research.google.com/github/gauravraidata/IITJ-projects/blob/main/AI_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
from queue import Queue

def initialize_warehouse(N, M, P, O, seed=42):
    random.seed(seed)
    warehouse = np.zeros((N, M), dtype=str)

    # Place obstacles
    obstacles = set()
    while len(obstacles) < O:
        x, y = random.randint(0, N-1), random.randint(0, M-1)
        if (x, y) not in obstacles:
            obstacles.add((x, y))
            warehouse[x, y] = 'X'  # Obstacle

    # Place packages and drop-offs
    package_locations = []
    dropoff_locations = set()
    while len(package_locations) < P:
        px, py = random.randint(0, N-1), random.randint(0, M-1)
        if (px, py) not in obstacles and (px, py) not in package_locations:
            package_locations.append((px, py))
            warehouse[px, py] = 'P'  # Package

    while len(dropoff_locations) < P:
        dx, dy = random.randint(0, N-1), random.randint(0, M-1)
        if (dx, dy) not in obstacles and (dx, dy) not in package_locations and (dx, dy) not in dropoff_locations:
            dropoff_locations.add((dx, dy))
            warehouse[dx, dy] = 'D'  # Drop-off point

    dropoff_locations = list(dropoff_locations)

    # Set robot start position
    Sx, Sy = 0, 0  # Fixed starting point
    warehouse[Sx, Sy] = 'S'

    return warehouse, package_locations, dropoff_locations, obstacles, (Sx, Sy)

def display_warehouse(warehouse):
    print("\nWarehouse Grid:")
    for row in warehouse:
        print(" ".join(cell if cell else '.' for cell in row))

def bfs_search(start, goal, obstacles, N, M):
    queue = Queue()
    queue.put((start, []))
    visited = set()

    while not queue.empty():
        (x, y), path = queue.get()

        if (x, y) == goal:
            return path + [(x, y)]

        if (x, y) in visited:
            continue
        visited.add((x, y))

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < N and 0 <= ny < M and (nx, ny) not in obstacles and (nx, ny) not in visited:
                queue.put(((nx, ny), path + [(x, y)]))

    return []  # No path found

def main():
    N, M, P, O = 6, 6, 3, 5  # Example parameters
    warehouse, packages, dropoffs, obstacles, start = initialize_warehouse(N, M, P, O)
    display_warehouse(warehouse)

    total_cost = 0
    reward = 0

    for i in range(P):
        pickup = packages[i]
        dropoff = dropoffs[i]

        path_to_pickup = bfs_search(start, pickup, obstacles, N, M)
        path_to_dropoff = bfs_search(pickup, dropoff, obstacles, N, M)

        if path_to_pickup and path_to_dropoff:
            total_cost += len(path_to_pickup) + len(path_to_dropoff)
            reward += 10  # Delivery reward
            start = dropoff  # Update robot's position

    final_score = reward - total_cost

    print("\nTotal Cost:", total_cost)
    print("Total Reward:", reward)
    print("Final Score:", final_score)

if __name__ == "__main__":
    main()
